# Testing Out Neural Network Regression

## Importing Necessary Libraries

In [68]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.optimizers import Adam
from keras.layers import Dense, LeakyReLU
from keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

## Loading in the Dataset

In [69]:
csv_url = 'https://gist.github.com/NickyDCFP/d675b176350f6a1d54455ffc35e350f9/raw/' + \
          '220fdbc2aec82b8b3e33681e4465813c02b5fccf/Spotify_Youtube.csv'
csv_df = pd.read_csv(csv_url)
# This column may not be necessary (duplicate of index), may remove
csv_df.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)
csv_df.head()

,ID,Artist,Url_spotify,Track,Album,Album_type,Uri,Danceability,Energy,Key,...,Url_youtube,Title,Channel,Views,Likes,Comments,Description,Licensed,official_video,Stream
0,0,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Feel Good Inc.,Demon Days,album,spotify:track:0d28khcov6AiegSCpG5TuT,0.818,0.705,6.0,...,https://www.youtube.com/watch?v=HyHNuVaZJ-k,Gorillaz - Feel Good Inc. (Official Video),Gorillaz,693555221.0,6220896.0,169907.0,Official HD Video for Gorillaz' fantastic trac...,True,True,1.040235e+09
1,1,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Rhinestone Eyes,Plastic Beach,album,spotify:track:1foMv2HQwfQ2vntFf9HFeG,0.676,0.703,8.0,...,https://www.youtube.com/watch?v=yYDmaexVHic,Gorillaz - Rhinestone Eyes [Storyboard Film] (...,Gorillaz,72011645.0,1079128.0,31003.0,The official video for Gorillaz - Rhinestone E...,True,True,3.100837e+08
2,2,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,New Gold (feat. Tame Impala and Bootie Brown),New Gold (feat. Tame Impala and Bootie Brown),single,spotify:track:64dLd6rVqDLtkXFYrEUHIU,0.695,0.923,1.0,...,https://www.youtube.com/watch?v=qJa-VFwPpYA,Gorillaz - New Gold ft. Tame Impala & Bootie B...,Gorillaz,8435055.0,282142.0,7399.0,Gorillaz - New Gold ft. Tame Impala & Bootie B...,True,True,6.306347e+07
3,3,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,On Melancholy Hill,Plastic Beach,album,spotify:track:0q6LuUqGLUiCPP1cbdwFs3,0.689,0.739,2.0,...,https://www.youtube.com/watch?v=04mfKJWDSzI,Gorillaz - On Melancholy Hill (Official Video),Gorillaz,211754952.0,1788577.0,55229.0,Follow Gorillaz online:\nhttp://gorillaz.com \...,True,True,4.346636e+08
4,4,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Clint Eastwood,Gorillaz,album,spotify:track:7yMiX7n9SBvadzox8T5jzT,0.663,0.694,10.0,...,https://www.youtube.com/watch?v=1V_xRb0x9aw,Gorillaz - Clint Eastwood (Official Video),Gorillaz,618480958.0,6197318.0,155930.0,The official music video for Gorillaz - Clint ...,True,True,6.172597e+08


In [70]:
irrelevant_columns = ["Url_spotify", "Uri", "Url_youtube", "ID", "Artist", "Track", "Album", "Album_type", "Title", "Channel", "Description", 'Licensed', 'official_video'] # For now, dropping all non-numeric columns
df = csv_df.drop(irrelevant_columns, axis=1)
df.head()

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,Views,Likes,Comments,Stream
0,0.818,0.705,6.0,-6.679,0.1770,0.008360,0.002330,0.6130,0.772,138.559,222640.0,693555221.0,6220896.0,169907.0,1.040235e+09
1,0.676,0.703,8.0,-5.815,0.0302,0.086900,0.000687,0.0463,0.852,92.761,200173.0,72011645.0,1079128.0,31003.0,3.100837e+08
2,0.695,0.923,1.0,-3.930,0.0522,0.042500,0.046900,0.1160,0.551,108.014,215150.0,8435055.0,282142.0,7399.0,6.306347e+07
3,0.689,0.739,2.0,-5.810,0.0260,0.000015,0.509000,0.0640,0.578,120.423,233867.0,211754952.0,1788577.0,55229.0,4.346636e+08
4,0.663,0.694,10.0,-8.627,0.1710,0.025300,0.000000,0.0698,0.525,167.953,340920.0,618480958.0,6197318.0,155930.0,6.172597e+08


In [71]:
y_cols = ['Views', 'Likes', 'Comments']
# df = df.dropna(subset=y_cols), 
# dropping all nan values as a stopgap for now
df = df.dropna()
df = df.loc[df['Views'] != 0]
num_likes = np.array(df['Likes']).astype(int)
num_comments = np.array(df['Comments']).astype(int)
num_views = np.array(df['Views']).astype(int)

In [72]:
X_df = df.drop(y_cols, axis=1)
X = np.array(X_df).astype('float32')
y = (num_likes / num_views).astype('float32') # Can also use comment proportion as target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(X_train.shape)

(14661, 12)


In [73]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [74]:
def r2(y_true, y_pred):
    score = tf.py_function(r2_score, (y_true, y_pred), tf.float64)
    return score


In [75]:
# Training r^2 is quite a bit better than the test r^2. Are probably overfitting.
l1_lambda = 0.1
l2_lambda = 0.1
model = keras.Sequential([
    Dense(5, activation='sigmoid', kernel_regularizer=regularizers.l1(l1_lambda), input_shape=(X_train_scaled.shape[1], )),
    Dense(5, activation='sigmoid', kernel_regularizer=regularizers.l2(l2_lambda)),
    Dense(1)
]) # Initial shape that I tried was this, but layers had 15 neurons per, and it was just all over the place, insane loss
# Even dropout didn't help. Could try out dropout I guess, but on 3 neurons per layer I'm not sure if it'll help. L1/L2 regularization probably helps more.

# model = keras.Sequential([
#     Dense(3, activation=LeakyReLU(alpha=0.01), input_shape=(X_train_scaled.shape[1], )),
#     Dense(3, activation=LeakyReLU(alpha=0.01)),
#     Dense(1)
# ]) Leaky ReLU had higher error :(

optimizer = Adam(learning_rate=0.0001) # Tried out 0.01 learning rate, not much different. 0.001 is the default. Lowering learning rate gives steady descent, nice
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[r2])

In [76]:
fit = model.fit(X_train_scaled, y_train,
                epochs=150,
                batch_size=150)

Epoch 1/150
98/98 [==============================] - 0s 1ms/step - loss: 2.7291 - r2: -4628.7529
Epoch 2/150
98/98 [==============================] - 0s 1ms/step - loss: 2.5923 - r2: -3686.0872
Epoch 3/150
98/98 [==============================] - 0s 1ms/step - loss: 2.4665 - r2: -3158.6948
Epoch 4/150
98/98 [==============================] - 0s 1ms/step - loss: 2.3492 - r2: -2638.0767
Epoch 5/150
98/98 [==============================] - 0s 1ms/step - loss: 2.2404 - r2: -2204.8027
Epoch 6/150
98/98 [==============================] - 0s 1ms/step - loss: 2.1384 - r2: -1825.7538
Epoch 7/150
98/98 [==============================] - 0s 1ms/step - loss: 2.0419 - r2: -1518.2319
Epoch 8/150
98/98 [==============================] - 0s 1ms/step - loss: 1.9509 - r2: -1247.9667
Epoch 9/150
98/98 [==============================] - 0s 1ms/step - loss: 1.8647 - r2: -1048.9124
Epoch 10/150
98/98 [==============================] - 0s 1ms/step - loss: 1.7830 - r2: -840.2737
Epoch 11/150
98/98 [==========

In [77]:
loss, r_2 = model.evaluate(X_test_scaled, y_test)
print(f'MSE: {loss}\nR^2:{r_2}')

153/153 [==============================] - 0s 2ms/step - loss: 1.9491e-04 - r2: -0.0519
MSE: 0.00019491053535602987
R^2:-0.05185729265213013


In [78]:
# Will probably shift into a different file later
from sklearn.feature_selection import mutual_info_regression
scores = mutual_info_regression(X_train_scaled, y_train)
for i in range(len(scores)):
    print(f"Feature {X_df.columns[i]} has a score of {scores[i]}")

Feature Danceability has a score of 0.10270233161683429
Feature Energy has a score of 0.10774695981217075
Feature Key has a score of 0.07989200751261993
Feature Loudness has a score of 0.10614310573219488
Feature Speechiness has a score of 0.1338227115301711
Feature Acousticness has a score of 0.12725996101853188
Feature Instrumentalness has a score of 0.06851475619172742
Feature Liveness has a score of 0.10155028401464428
Feature Valence has a score of 0.11778418113043987
Feature Tempo has a score of 0.11655010259371679
Feature Duration_ms has a score of 0.1691226449744816
Feature Stream has a score of 0.1837925784255967


In [79]:
y

array([0.00896958, 0.01498547, 0.03344874, ..., 0.00922965, 0.01347008,
       0.01565247], dtype=float32)